<a href="https://colab.research.google.com/github/chrbertsch/fmi3-features/blob/main/adjoint_derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adjoint Derivatives

This notebook demonstrates how to efficiently retrieve [adjoint derivative](https://fmi-standard.org/docs/3.0/#partial-derivatives) using the `fmi3GetAdjointDerivative()` function introduced in FMI 3.0.
We'll use the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) model from the [Reference FMUs](https://github.com/modelica/Reference-FMUs) that implements the following equation:

```
der(x0) = x1
der(x1) = mu * ((1 - x0 * x0) * x1) - x0
```

In [ ]:
!uv pip install fmpy[complete] --system
!git clone https://github.com/chrbertsch/fmi3-features

In [ ]:
from fmpy import *
from shutil import rmtree
import plotly.graph_objects as go
import numpy as np


fmu_filename = 'fmi3-features/resources/VanDerPol.fmu'

# change the start values, so we can see how the states converge
result = simulate_fmu(fmu_filename, start_values={'x0': 0, 'x1': 4})

fig = go.Figure()

# plot the trajectory of the two states
fig.add_trace(go.Scatter(x=result['x0'], y=result['x1']))

axes_attrs = dict(showgrid=True, gridwidth=1, ticklen=0, gridcolor='LightGrey', linecolor='black', showline=True, zerolinewidth=1, zerolinecolor='LightGrey')
fig.update_xaxes(**axes_attrs)
fig.update_yaxes(**axes_attrs)
fig.update_layout(margin=dict(t=30, b=0, r=30, l=0), plot_bgcolor='rgba(0,0,0,0)', xaxis_title="x0", yaxis_title="x1")

fig.show()

In [ ]:
# extract the FMU
unzipdir = extract(fmu_filename)

# load the model description
model_description = read_model_description(unzipdir)

# build a dictionary of (variable_name -> value_reference)
vr = dict((v.name, v.valueReference) for v in model_description.modelVariables)

# instantiate the FMU
fmu_instance = instantiate_fmu(unzipdir=unzipdir,
                               model_description=model_description,
                               fmi_call_logger=print,     # print FMI calls to the console
                               fmi_type='ModelExchange')  # workaround for https://github.com/modelica/Reference-FMUs/issues/245

# initialize the FMU instance
fmu_instance.enterInitializationMode()
fmu_instance.exitInitializationMode()

In [ ]:
knowns   = [vr['x0'],      vr['x1']     ]  # value references of the continuous states
unknowns = [vr['der(x0)'], vr['der(x1)']]  # value references of the continuous state derivatives
seed     = [1, 1]                          # dummy seed values (typically provided by algorithm)

# create a 2x2 matrix of zeros
J = np.zeros((len(knowns), len(knowns)))

# construct the Jacobian matrix column wise (FMI 2.0 method)
for i, known in enumerate(knowns):
    J[:, i] = fmu_instance.getDirectionalDerivative(unknowns, [known], [1])

# calculate v^T * J
sensitivity = seed @ J

sensitivity

In [ ]:
# retrive v^T * J with a single call
sensitivity = fmu_instance.getAdjointDerivative(unknowns, knowns, seed)

sensitivity

In [ ]:
# clean up
fmu_instance.freeInstance()
rmtree(unzipdir)